In [ ]:
#@title نصب بهینه شده
!git clone https://github.com/yaranbarzi/roop
%cd roop
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_GPU_MEMORY_GROWTH'] = 'true'

!pip install onnxruntime-gpu==1.15.1 -q
!pip install -r requirements.txt -q
!pip install ipywidgets -q
!pip uninstall -y jax jaxlib tensorflow -q
!pip install tensorflow==2.13.0 -q
!pip install jax==0.4.20 jaxlib==0.4.20 -q

import gc
gc.collect()

from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

print("✅ Setup complete with GPU optimization. You can now proceed to the next steps.")

In [ ]:
#@title آپلود عکس
%cd /content/roop

# Clean up previous uploads
if os.path.exists("source.png"):
    os.remove("source.png")

uploaded = files.upload()

if uploaded:
    # Get the filename of the uploaded file
    original_filename = next(iter(uploaded))
    # Rename and save it as source.png
    os.rename(original_filename, "source.png")
    print("✅ Source image uploaded and saved as 'source.png'")
else:
    print("⚠️ No file uploaded.")

In [ ]:
#@title آپلود ویدیوی هدف
%cd /content/roop

# Clean up previous uploads
if os.path.exists("target.mp4"):
    os.remove("target.mp4")

uploaded = files.upload()

if uploaded:
    # Get the filename of the uploaded file
    original_filename = next(iter(uploaded))
    # Rename and save it as target.mp4
    os.rename(original_filename, "target.mp4")
    print("✅ Target video uploaded and saved as 'target.mp4'")
else:
    print("⚠️ No file uploaded.")

In [ ]:
#@title اجرای دیپ فیک بهینه شده

enable_enhancer = True #@param {type:"boolean"}

face_selection_mode = "Swap Default Face (Fastest)" #@param ["Swap Default Face (Fastest)", "Swap ALL faces in video", "Swap a SPECIFIC face"]
specific_face_number = 0 #@param {type:"number"}

import os
from google.colab import files
from IPython.display import display, HTML
import gc

%cd /content/roop

# آزادسازی حافظه قبل از شروع
gc.collect()

# ساخت command بهینه شده
command = (
    "python run.py -s 'source.png' -t 'target.mp4' -o 'output.mp4' "
    "--keep-fps --output-video-quality 1 --execution-provider cuda "
    "--max-memory 8 --output-video-encoder libx264 "
)

# اضافه کردن پردازشگرها بر اساس انتخاب کاربر
if enable_enhancer:
    command += "--frame-processor face_swapper face_enhancer "
else:
    command += "--frame-processor face_swapper "

# تنظیم حالت انتخاب صورت
if "ALL faces" in face_selection_mode:
    command += "--many-faces "
elif "SPECIFIC face" in face_selection_mode:
    command += f"--reference-face-position {specific_face_number} "

print("""
Final optimized command to be executed:""")
print(f"""
{command}""")
print("""
--- Starting optimized process... ---""")

# اجرای command
!{command}

# آزادسازی حافظه بعد از پردازش
gc.collect()

print("""
--- ✅ Process finished! ---""")

if os.path.exists("output.mp4"):
    print("""
Downloading the final video...""")
    files.download("output.mp4")
else:
    print("""
⚠️ Output file 'output.mp4' not found. Did the process run correctly?""")

# نمایش پروموشن
image_url = 'https://huggingface.co/Toolsai/dubtest/resolve/main/newgolden.png'
youtube_channel_url = 'https://youtube.com/@aigolden'

html_code = f'''
<div style="text-align: center; border: 2px solid #e0e0e0; padding: 15px; border-radius: 12px; background-color: #f9f9f9; max-width: 350px; margin: auto;">
    <a href="{youtube_channel_url}" target="_blank" title="رفتن به کانال یوتیوب AIGOLDEN">
        <img src="{image_url}" alt="AIGOLDEN YouTube Channel" style="max-width: 100%; height: auto; border-radius: 8px;">
    </a>
    <p style="font-size: 16px; font-family: 'Vazir', sans-serif; margin-top: 15px; color: #333;">
        برای مشاهده آموزش‌های بیشتر، ما را در یوتیوب دنبال کنید.
    </p>
    <a href="{youtube_channel_url}" target="_blank" style="text-decoration: none; display: inline-block; background-color: #FF0000; color: white; padding: 10px 20px; border-radius: 8px; font-weight: bold; font-family: 'Vazir', sans-serif; margin-top: 10px;">
        🚀 دنبال کردن در یوتیوب
    </a>
</div>
'''

display(HTML(html_code))